<a href="https://colab.research.google.com/github/mzucali/pyMinColab/blob/main/ExtractFromSummaryEMPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LEGGE IL FILE SUMMARY.TXT ed estrae DL e SD:
>input il file summary.txt o qualunque altro nome MA .TXT

In [1]:
# load and read summary file from EMPA
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving 14-7-11summary.txt to 14-7-11summary.txt
User uploaded file "14-7-11summary.txt" with length 446983 bytes


In [4]:
num_elements = input("QUANTI ELEMENTI ANALIZZATI? ")
#num_elements = 11
loop = int(num_elements)+2

QUANTI ELEMENTI ANALIZZATI? 11


In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 14 11:30:45 2023
GRAZIE ALESSANDRO COMUNIAN
@author: alex
"""

#file_in = open("summary.txt", "r")
file_in = open(fn,"r")
outFile = open("result2.txt", "w")

lines = file_in.readlines()

for i, line in enumerate(lines):
    line = line.replace("\t\t","\t")
    line = line.replace("  ","")
    line_s = line.split()
    if line_s:
        try:
            if(line_s[4] == "Comment"):
                # Keep track of the comment value
                comm = line_s[6]
                # Just a spacing line
                print()
        except IndexError:
            pass
        
        if(line_s[0] == "Curr.(A)"):
#        if(line_s[0] == "Element	"):  
            print("ID {0} Comment".format(lines[i+1][:-1]))

            outFile.write("ID {0} Comment\n".format(lines[i+1][:-1]))
            for j in range(2,loop):
                print("{0} {1}\n".format(lines[i+j][:-1], comm), end="")
                outFile.write("{0} {1}\n".format(lines[i+j][:-1], comm))
                #outFile.write(" {0}\n".format(lines))

file_in.close()
outFile.close() #result2.txt




ID Element	Peak(mm)	Net(cps)	Bg-	Bg+	S.D.(%)	D.L.(ppm)	  Comment
1	Mg	107.988	-0.2	6.8	4.4	100.00		197	  grossularia-std
2	Na	129.506	-0.2	1.6	0.8	100.00		179	  grossularia-std
3	Ti	88.090	0.5	7.3	5.8	150.41		278	  grossularia-std
4	Mn	146.262	2.2	3.9	5.2	30.80	283	  grossularia-std
5	K	119.614	0.2	12.0	11.3	395.81		102	  grossularia-std
6	Al	91.151	1111.3	12.1	11.4	0.56	216	  grossularia-std
7	Si	77.281	1638.1	6.5	7.2	0.45	190	  grossularia-std
8	Cr	73.248	0.6	13.5	8.4	158.99		310	  grossularia-std
9	Fe	134.707	13.3	6.0	5.2	7.17	291	  grossularia-std
10	Ca	107.316	3818.0	21.5	22.2	0.30	146	  grossularia-std
11	Ni	115.317	0.3	6.2	8.5	278.67		364	  grossularia-std

ID Element	Peak(mm)	Net(cps)	Bg-	Bg+	S.D.(%)	D.L.(ppm)	  Comment
1	Mg	107.988	1.0	5.5	4.5	65.38		190	  grossularia-std
2	Na	129.506	-0.1	1.7	1.3	100.00		195	  grossularia-std
3	Ti	88.090	1.2	8.5	4.6	60.98		269	  grossularia-std
4	Mn	146.262	2.2	5.2	4.3	31.80	289	  grossularia-std
5	K	119.614	-2.3	15.1	12.1	100.00		110	  gros

In [7]:
#RESULT3 con spazi e tab corretti
newopen = open("result3.txt", "w")
with open("result2.txt") as f:
  first_line = f.readline()
  first_line_split = first_line.split()
  first_line = '\t'.join(first_line_split)+'\n'
  newopen.write(first_line)
  print(first_line)

with open("result2.txt") as f:
  for line in f:
    if 'ID Element	Peak(mm)' not in line:
      line_split = line.split()
      line = '\t'.join(line_split)+'\n'  
      newopen.write(line)     
newopen.close()

ID	Element	Peak(mm)	Net(cps)	Bg-	Bg+	S.D.(%)	D.L.(ppm)	Comment



RESULT3.TXT con dati estratti di SD e DL

In [6]:
# EXTRACT X and Y coodinates
file_in2 = open(fn, "r")
lines = file_in2.readlines()
names = []
Xs = []
Ys = []

for i, line in enumerate(lines):
    line_s = line.split()
    #print(line_s)
    if line_s:
        try:
            if(line_s[4] == "Comment"):
                # Keep track of the comment value
                name = line_s[6]
                names.append(name)
                # Just a spacing line
            elif(line_s[0] == "Stage"):
                # Keep track of the comment value
                x = line_s[3]
                Xs.append(x)
                y = line_s[5]
                Ys.append(y)
                # Just a spacing line
        except IndexError:
            pass

import pandas as pd

print("Sample: ",len(names))
print("X: ", len(Xs))
print("Y: ", len(Ys))
out = pd.DataFrame(
    {'Sample': names, 
     'X':Xs, 
     'Y':Ys
     })
#list(zip(names, Xs, Ys)))
print(out)
out.to_csv('coordinate.csv', index=False)

Sample:  209
X:  209
Y:  209
                 Sample        X        Y
0       grossularia-std  32.6205  28.4405
1       grossularia-std  32.6205  28.4405
2             ZR19-C1-1  17.3380  92.1435
3             ZR19-C1-2  16.9085  91.9280
4    HORNEBLENDA123-STD  73.8125  29.6915
..                  ...      ...      ...
204      FZB09063-C5-65  41.6420  70.7670
205      FZB09063-C5-66  41.6620  70.7955
206      FZB09063-C5-67  41.6920  70.7670
207      FZB09063-C5-68  41.7010  70.7700
208      FZB09063-C5-69  41.7240  70.7695

[209 rows x 3 columns]


COORDINATE.CSV con sample name e coordinate X e Y

TUTTO DA **SCARICARE**